In [ ]:
import sys
sys.path.append("C:/Users/Tobias.Turke/Desktop/Uni/GraphToSequenceNN/Scripts/")
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import multiply
#from keras.models import Model
#from keras.callbacks import History
from NetworkHandler.KerasSupportMethods.SupportMethods import ConsoleReporter
from NetworkHandler.KerasSupportMethods.ControledBasicOperations import ControledTensorOperations as CTO
from NetworkHandler.Neighbouring.NeighbourhoodCollector import Neighbourhood as Nhood

#These are the edges between the nodes.
current = np.array( [
                    [0,0.3,0.3,1,0.3],
                    [0.1,0.3,1,0,0.2],
                    [0,0.7,1,1,0.4],
                    [0.8,0.2,1,1,0.2]
                    ], dtype='float32')

current_ten = K.variable (value=current, dtype='float32')

################################

neigbours = np.array([
                     [0,1,1,1],
                     [0,0,0,1],
                     [0,0,0,0],
                     [0,0,0,0]
                     ], dtype='float32')

neighbouring = K.variable (value=neigbours, dtype='float32')

################################

concate_multiplier = 2
in_dim = current_ten.shape[1] + neighbouring.shape[1]
out_dim = 5

'''
weight_array = np.random.rand(in_dim,out_dim)
'''

weight_array = np.array( 
    [[0.66966766, 0.5902943,  0.07666955, 0.5140854,  0.15426576],
     [0.05736156, 0.5055818,  0.20662808, 0.05926343, 0.07687023],
     [0.98930365, 0.4705559,  0.474547,   0.09166889, 0.26834506],
     [0.87441427, 0.70664155, 0.5362093,  0.8938229,  0.63828397],
     [0.08835044, 0.10014847, 0.51245475, 0.3310912,  0.8174404 ],
     [0.57453996, 0.0329449,  0.9358181,  0.25454348, 0.33277592],
     [0.70978075, 0.31889898, 0.76369053, 0.48174727, 0.08540577],
     [0.70978075, 0.31889898, 0.76369053, 0.48174727, 0.08540577],
     [0.88373375, 0.02076262, 0.16413115, 0.8409406,  0.98415136]])

weights_ten = K.variable (value=weight_array, dtype='float32')
################################

zero_cc_array = np.zeros(current_ten.shape[0])
zero_cc_ten = K.variable (value=zero_cc_array, dtype='float32')

################################

bias_array = np.zeros(out_dim)
bias_ten = K.variable (value=bias_array, dtype='float32')

################################
  

sess = tf.Session()
K.set_session(sess)
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
with sess:
    init_g.run()
    init_l.run()
    
    
    
    ConsoleReporter('Inits',current_ten,True)
    ConsoleReporter('Neighbouring',neighbouring,True)
    ConsoleReporter('weights_ten',weights_ten,True)
    ConsoleReporter('bias_ten',bias_ten,True)
    ConsoleReporter('zero_cc_ten',zero_cc_ten,True)
    
    agg_neigh_vecs = Nhood(current_ten, neighbouring, aggregator='mean').Execute()
    ConsoleReporter('agg_neigh_vecs',agg_neigh_vecs,True)
    
    concatenated = CTO.ControledConcatenation(current_ten, agg_neigh_vecs)
    ConsoleReporter('concatenated',concatenated,True)
    
    difference = CTO.ControledShapeDifference(weights_ten, concatenated)
    print('difference:\n',difference, '\n')
    
    extended = CTO.ControlledMatrixExtension(concatenated, zero_cc_ten, difference)
    ConsoleReporter('extended',extended,True)
    
    multiplied = CTO.ControledWeightDotProduct(extended, weights_ten)
    ConsoleReporter('multiplied',multiplied,True)
    
    biased = CTO.ControledBiased(multiplied, bias_ten)
    ConsoleReporter('biased',biased,True)
